In [2]:
import os
import sys
import json
import requests
import pandas as pd

def add_modules():
    """
    Starting at the current directory and proceeding up the file system
    tree, search for a directory named `modules`.  If found, and if not
    already there, add to the Python module search path.
    
    Params: None
    
    Return: None
    """
    directory = "."
    levels = 0
    while not os.path.isdir(os.path.join(directory, "modules")) and \
          levels < 5:
        directory = os.path.join(directory, "..")
        levels += 1
    module_path = os.path.abspath(os.path.join(directory, "modules"))
    if os.path.isdir(module_path):
        if not module_path in sys.path:
            sys.path.append(module_path)

add_modules()
import util

In [3]:
def buildURL(isbnNum):
    """
    This function builds a url that can be used to get the information of the book identified by the ISBN 
    
    Parameters: isbnNum, a string containing an ISBN for a certain book
    
    Returns: u, a string of the url
    """
    u = f"https://api2.isbndb.com/book/{isbnNum}"
    return u

In [4]:
def buildRequest(apikey, isbnNum):
    """
    This function returns a dictionary of the results processed by the url
    
    Parameters: isbnNum, a string containing an ISBN for a certain book
                apiKey, a string holding the apikey in order to authorize the request
    
    Returns: A json object with all of the information about the book identified by the ISBN
    """
    headD = {
        "accept": "application/json",
        "Authorization": apikey
    }
    url = buildURL(isbnNum)
    
    r = requests.get(url, headers=headD)
    return r.json()

In [23]:
def getPublisher(apikey, isbn):
    """
    This function gets the publisher from the json object and returns None if there is no publisher listed
    
    Parameters: isbn, a string containing an ISBN for a certain book
                apiKey, a string holding the apikey in order to authorize the request
                
    Returns: The publisher if there is one, None if otherwise
    """
    r = buildRequest(apikey, isbn)
    try: 
        r['book']['publisher']
    except:
        return None
    return r['book']['publisher']

In [41]:
def addPublisher(apikey, dataFrame):
    """
    This function creates a new dataframe and adds a column of publishers for each isbn number in the given dataframe
    
    Parameters: apiKey, a string holding the apikey in order to authorize the request
                dataFrame, a pandas dataframe that contains a column called 'ISBN'
                
    Returns: A new dataframe with an additional column called Publisher
    """
    newDF = dataFrame
    isbns = dataFrame['ISBN']
    pubList = []
    for num in isbns:
        pub = getPublisher(apikey, num)
        pubList.append(pub)
        
    newDF['Publisher'] = pubList
    return newDF

In [54]:
def createCSV(table, filename, datadir):
    """
    This function creates a csv file names filename in datadir
    
    Parameters: table, a pandas dataframe
                filename, a string for the name of the output file
                datadir, a string containing the directory where the file should be placed
    
    Returns: None
    """
    table.to_csv(path_or_buf=os.path.join(datadir, filename), index=False)

In [55]:
def main():
    isbnKey = util.read_creds("ISBN", ".", "Bookcreds.json")
    myapikey = isbnKey['api-key']
    nyt = pd.read_csv("New York Times Book List.csv")
    newNYT = addPublisher(myapikey, nyt)
    createCSV(newNYT, "New York Times Publishers.csv", os.getcwd())

In [56]:
main()